In [ ]:
#web scraping for data
#to build database of windows for training
import requests, lxml, re, json, urllib
from bs4 import BeautifulSoup

headers = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

params = {
    "q": "window hdb",
    "tbm": "isch",
    "ijn": "0",
}

html = requests.get("https://www.google.com/search", params=params, headers=headers)
soup = BeautifulSoup(html.text, 'lxml')


def get_images_data():

    all_script_tags = soup.select('script')

    # # https://regex101.com/r/48UZhY/4
    matched_images_data = ''.join(re.findall(r"AF_initDataCallback\(([^<]+)\);", str(all_script_tags)))

    matched_images_data_fix = json.dumps(matched_images_data)
    matched_images_data_json = json.loads(matched_images_data_fix)

    # https://regex101.com/r/pdZOnW/3
    matched_google_image_data = re.findall(r'\[\"GRID_STATE0\",null,\[\[1,\[0,\".*?\",(.*),\"All\",', matched_images_data_json)

    # https://regex101.com/r/NnRg27/1
    matched_google_images_thumbnails = ', '.join(
        re.findall(r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]',
                   str(matched_google_image_data))).split(', ')

    print('Google Image Thumbnails:')  # in order
    for fixed_google_image_thumbnail in matched_google_images_thumbnails:
        # https://stackoverflow.com/a/4004439/15164646 comment by Frédéric Hamidi
        google_image_thumbnail_not_fixed = bytes(fixed_google_image_thumbnail, 'ascii').decode('unicode-escape')

        # after first decoding, Unicode characters are still present. After the second iteration, they were decoded.
        google_image_thumbnail = bytes(google_image_thumbnail_not_fixed, 'ascii').decode('unicode-escape')
        print(google_image_thumbnail)
        

    # removing previously matched thumbnails for easier full resolution image matches.
    removed_matched_google_images_thumbnails = re.sub(
        r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]', '', str(matched_google_image_data))

    # https://regex101.com/r/fXjfb1/4
    # https://stackoverflow.com/a/19821774/15164646
    matched_google_full_resolution_images = re.findall(r"(?:'|,),\[\"(https:|http.*?)\",\d+,\d+\]",
                                                       removed_matched_google_images_thumbnails)


    print('\nFull Resolution Images:')  # in order
    count = 12
    for index, fixed_full_res_image in enumerate(matched_google_full_resolution_images):
        original_size_img_not_fixed = bytes(fixed_full_res_image, 'ascii').decode('unicode-escape')
        original_size_img = bytes(original_size_img_not_fixed, 'ascii').decode('unicode-escape')
        print(original_size_img)

        # ------------------------------------------------
        # Download original images

        # print(f'Downloading {index} image...')

        opener=urllib.request.build_opener()
        opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582')]
        urllib.request.install_opener(opener)

        urllib.request.urlretrieve(original_size_img, f'./window-img/window-{count}.jpg')
        count += 1



get_images_data()

In [ ]:
#load manual labelled data for windows
from roboflow import Roboflow
rf = Roboflow(api_key="XIkFjSCMqTJ3giSB84K3")
project = rf.workspace().project("window-detection")
dataset = project.version(1).download("yolov5")

In [ ]:
#train command for model
!python3 ./yolov5/train.py --data ./Window-Detection-1/data.yaml --weights yolov5s.pt --img 640

In [17]:
#run on image
#source accepts folder of images or an image
file = './actual-data/actual-window-3.jpg'
name = file.split('/')[-1].strip('.jpg')
!python3 ./yolov5/detect.py --weights ./yolov5/runs/train/exp/weights/best.pt \
                            --img 640 --conf 0.7 --source ./actual-data/actual-window-3.jpg --save-txt

#save-txt generates a text file of class, x, y, width, height in each row for each object
#assuming image is plotted into 1x1 graph 

detect: weights=['./yolov5/runs/train/exp/weights/best.pt'], source=./actual-data/actual-window-3.jpg, imgsz=[640, 640], conf_thres=0.7, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-127-g554f782 torch 1.10.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /Users/yanch/Documents/personal/uni/fintech/project-2/actual-data/actual-window-3.jpg: 480x640 1 windows, Done. (0.256s)
Speed: 7.0ms pre-process, 256.5ms inference, 19.6ms NMS per image at shape (1, 3, 640, 640)
Results saved to yolov5/runs/detect/exp8
1 labels saved to yolov5/runs/detect/exp8/labels


In [4]:
file = './actual-data/actual-window-3.jpg'
name = file.split('/')[-1].strip('.jpg')
print(name)

actual-window-3


In [ ]:
#width and height (pixels) data is available in .txt
#how to get pixels/metric? => reference object in image

ppm = None #pixels per metric

#find reference object (new class) and its width and height in txt file
#convert to ppm
#use ppm to calculate window width and height

In [15]:
#alternative
#with specified distance from object and assumed camera parameters, we can find the object's real height

#formula: (object pixel height * pixel size * distance)/focal length

#26mm focal length
#assuming distance from window is 3m

fl = 0.026 #in m
d = 3
psize = 0.000065
w = 0.403125 * 640
h = 0.359167 * 640
rw = w * psize * d / fl
rh = h * psize * d / fl
print(rw, rh)

#to use real photo to calibrate psize

1.935 1.7240016000000002
